In [1]:
import json
import time

from dotenv import load_dotenv
import os

from data_utils import get_followers_from_lists, DATA_PATH, get_scaraped_profiles, get_retweeters_repliers, get_author_ids, get_follow_from_lists
from twitter_utils import get_client, get_profiles

load_dotenv()  # take environment variables from .env.

True

In [2]:
client = get_client()

In [4]:
ids = get_followers_from_lists()

In [5]:
len(ids)

12439182

In [6]:
out_path = os.path.join(DATA_PATH, 'followers_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'followers_missing.jsonl')
already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

ids_ = [id_ for id_ in ids if id_ not in already_scraped]
print(f"{len(ids_)} users left to scrape")

12439182 users were already scraped
0 users left to scrape


In [45]:
errors = get_profiles(ids=ids_, client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 6/6 [00:02<00:00,  2.58it/s]


In [46]:
print(len(errors))

0


In [33]:
other_errors = get_profiles(ids=errors, client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 2/2 [00:00<00:00,  3.16it/s]


In [4]:
repliers = get_retweeters_repliers()
len(repliers)

8155

In [6]:
out_path = os.path.join(DATA_PATH, 'repliers_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'repliers_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

repliers = [id_ for id_ in repliers if id_ not in already_scraped]
print(f"{len(repliers)} users left to scrape")

follower_errors = get_profiles(ids=repliers, client=client, out_path=out_path, error_path=err_path)

8157 users were already scraped
8155 users left to scrape


batch: 100%|██████████| 82/82 [00:31<00:00,  2.61it/s]


In [7]:
len(follower_errors)

0

In [15]:
author_ids = set(get_author_ids(2016))
author_ids.update(get_author_ids(2020))
author_ids = list(sorted(author_ids))
print(len(author_ids))

1024


In [16]:
out_path = os.path.join(DATA_PATH, 'authors_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'author_profiles_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

author_ids = [id_ for id_ in author_ids if id_ not in already_scraped]
print(f"{len(ids_)} users left to scrape")

author_errors = get_profiles(ids=author_ids, client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 11/11 [00:04<00:00,  2.51it/s]


In [17]:
len(author_errors)

0

In [ ]:
'follower_lists_new.jsonl'
os.path.join(DATA_PATH, )

In [5]:
author_ids = get_follow_from_lists()
out_path = os.path.join(DATA_PATH, 'new_followers_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'new_followers_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

author_ids = [id_ for id_ in author_ids if id_ not in already_scraped]
print(f"{len(author_ids)} users left to scrape")

follower_errors = get_profiles(ids=author_ids, client=client, out_path=out_path, error_path=err_path)

0 users were already scraped
3759136 users left to scrape


batch: 100%|██████████| 37592/37592 [31:17:57<00:00,  3.00s/it]   


In [6]:
author_ids = get_follow_from_lists('followee_lists_of_new_retweeters_repliers.jsonl')
out_path = os.path.join(DATA_PATH, 'new_followees_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'new_followees_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

author_ids = [id_ for id_ in author_ids if id_ not in already_scraped]
print(f"{len(author_ids)} users left to scrape")

followee_errors = get_profiles(ids=author_ids, client=client, out_path=out_path, error_path=err_path)

0 users were already scraped
1590084 users left to scrape


batch: 100%|██████████| 15901/15901 [13:16:04<00:00,  3.00s/it]   


In [7]:
len(followee_errors), len(follower_errors)

(0, 0)

In [9]:
author_ids = get_follow_from_lists('followee_lists_new.jsonl')
out_path = os.path.join(DATA_PATH, 'new_followees_of_authors_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'new_followees_of_authors_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

author_ids = [id_ for id_ in author_ids if id_ not in already_scraped]
print(f"{len(author_ids)} users left to scrape")
import time
time.sleep(60*60*6)
followee_errors = get_profiles(ids=author_ids, client=client, out_path=out_path, error_path=err_path)

0 users were already scraped
12521 users left to scrape


batch: 100%|██████████| 126/126 [00:49<00:00,  2.57it/s]


In [3]:
author_ids = get_follow_from_lists('followee_lists_newer.jsonl')
out_path = os.path.join(DATA_PATH, 'newer_followees_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'newer_followees_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

author_ids = [id_ for id_ in author_ids if id_ not in already_scraped]
print(f"{len(author_ids)} users left to scrape")

follower_errors = get_profiles(ids=author_ids, client=client, out_path=out_path, error_path=err_path)

1290120 users were already scraped
699657 users left to scrape


batch: 100%|██████████| 6997/6997 [5:45:34<00:00,  2.96s/it]   


In [3]:
author_ids = get_follow_from_lists('follower_lists_newer.jsonl')
out_path = os.path.join(DATA_PATH, 'newer_followers_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'newer_followers_missing.jsonl')

already_scraped = get_scaraped_profiles(out_path, err_path)
print(f"{len(already_scraped)} users were already scraped")

author_ids = [id_ for id_ in author_ids if id_ not in already_scraped]
print(f"{len(author_ids)} users left to scrape")

follower_errors = get_profiles(ids=author_ids, client=client, out_path=out_path, error_path=err_path)

11561311 users were already scraped
0 users left to scrape


batch: 0it [00:00, ?it/s]


In [4]:
len(follower_errors)
follower_errors2 = get_profiles(ids=follower_errors, client=client, out_path=out_path, error_path=err_path)


batch: 0it [00:00, ?it/s]


In [5]:
len(follower_errors2)

0